In [ ]:


# 5. Predicting the prices
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# 6. Plot the results
train_plot = np.empty_like(data)
train_plot[:, :] = np.nan
train_plot[time_step:len(train_predict) + time_step, :] = train_predict

test_plot = np.empty_like(data)
test_plot[:, :] = np.nan
test_plot[len(train_predict) + (time_step * 2) + 1:len(data) - 1, :] = test_predict

plt.figure(figsize=(15, 6))
plt.plot(scaler.inverse_transform(scaled_data), label='Original Data')
plt.plot(train_plot, label='Train Prediction')
plt.plot(test_plot, label='Test Prediction')
plt.legend()
plt.show()

# Predict the next day and week prices
last_60_days = scaled_data[-60:]
X_input = last_60_days.reshape(1, -1)
X_input = X_input.reshape((1, time_step, 1))

# Next day prediction
next_day_price = model.predict(X_input)
next_day_price = scaler.inverse_transform(next_day_price)
print(f"Next day price prediction: {next_day_price[0][0]}")

# Next week prediction
next_week_prices = []
current_input = X_input

for _ in range(7):
    next_price = model.predict(current_input)
    next_week_prices.append(scaler.inverse_transform(next_price)[0][0])
    current_input = np.append(current_input[:, 1:, :], next_price.reshape(1, 1, 1), axis=1)

print(f"Next week price predictions: {next_week_prices}")

# 7. Brief report
print("\nBrief Report:")
print(f"Stock Symbol: {stock_symbol}")
print(f"Data Period: {stock_data.index.min()} to {stock_data.index.max()}")
print(f"GRU Model Architecture: {model.summary()}")
print(f"Train size: {train_size}, Test size: {test_size}")
print(f"Time step for training: {time_step}")


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

In [3]:
# 1. Load the stock data
stock_symbol = 'AAPL'  # Apple Inc. as an example
stock_data = yf.download(stock_symbol, start='2000-01-01')

# 2. Prepare the dataset
data = stock_data['Close'].values.reshape(-1, 1)

# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Prepare the training dataset
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Reshape input to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split into training and test sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


[*********************100%%**********************]  1 of 1 completed


In [4]:
# 3. Build the GRU model
model = Sequential()
model.add(GRU(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(GRU(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


In [5]:
# 4. Train the model
model.fit(X_train, y_train, batch_size=1, epochs=1)

4884/4884 [==============================] - 127s 25ms/step - loss: 4.7159e-05
